In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
import re
import math
import os

In [2]:
class Search_BigKinds:
    '''
    
    빅카인즈 크롤링 하는 클래스
    
    '''
    click_time = 1.5
    send_time = 0.5
    date_time = 2 # 날짜 변경해서 새로 driver를 켤 때 변경시간
    wait_time = 10 # 오류 발생시 기다리는 시간
    def __init__(self):
        '''
            기본 드라이브 실행
        '''
        # create a new chrome session
        prefs = {'profile.default_content_setting_values': {'cookies' : 2,
                         'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2,
                         'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2,
                         'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2,
                         'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2,
                         'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2,
                         'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}
        options = Options()
        # options.add_experimental_option('prefs', prefs) 
        # options.add_argument("start-maximized") 
        # options.add_argument("disable-infobars") 
        # options.add_argument("--disable-extensions")
        # options.add_argument('--headless')
        # options.add_argument('--no-sandbox')
        
        self.Crawling_news = dict({"Date":[],"Title":[],"Content":[]})
        
        try:
            self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()));sleep(self.date_time)
        except:
            sleep(self.wait_time)  
            self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()));sleep(self.date_time)
        self.driver.implicitly_wait(self.wait_time)
        self.driver.maximize_window()

        try:
            self.driver.get(url);sleep(self.date_time)
        except:
            sleep(self.wait_time)
            self.driver.get(url);sleep(self.date_time)
            

    def Search_keyword(self,keyword,start_day,end_day):
        self.keyword = keyword
        self.start_day = start_day
        self.end_day = end_day
        
        # 키워드 입력
        try:
            self.driver.find_element(By.CLASS_NAME, 'search-key').send_keys(keyword);sleep(self.send_time)
        except:
            sleep(self.wait_time)
            self.driver.find_element(By.CLASS_NAME, 'search-key').send_keys(keyword);sleep(self.send_time)

        # 상세 검색
        try:
            self.driver.find_element(By.CLASS_NAME, 'btn-srchDetail.btn-srchDetail-search').click();sleep(self.click_time)
        except:
            sleep(self.wait_time)
            self.driver.find_element(By.CLASS_NAME, 'btn-srchDetail.btn-srchDetail-search').click();sleep(self.click_time)
        
        # 기간 메뉴
        try:
            self.driver.find_element(By.CLASS_NAME, 'tab-btn.search-tab_group').click();sleep(self.click_time)
        except:
            sleep(self.wait_time)
            self.driver.find_element(By.CLASS_NAME, 'tab-btn.search-tab_group').click();sleep(self.click_time)
        
        # 시작 기간
        try:
            self.driver.find_element(By.ID, 'search-begin-date').send_keys("\b\b\b\b\b\b\b\b\b\b{}".format(start_day));sleep(self.send_time)
        except:
            sleep(self.wait_time)
            self.driver.find_element(By.ID, 'search-begin-date').send_keys("\b\b\b\b\b\b\b\b\b\b{}".format(start_day));sleep(self.send_time)
            
        # 끝 기간
        try:
            self.driver.find_element(By.ID, 'search-end-date').send_keys("\b\b\b\b\b\b\b\b\b\b{}".format(end_day));sleep(self.send_time)
        except:
            sleep(self.wait_time)
            self.driver.find_element(By.ID, 'search-end-date').send_keys("\b\b\b\b\b\b\b\b\b\b{}".format(end_day));sleep(self.send_time)
        
        self.driver.find_element(By.CLASS_NAME, 'btn-srch.input-group-btn.news-search-btn').click();sleep(self.click_time)
            
        try:
            self.total_cnt = int(re.sub(',','',self.driver.find_element(By.CLASS_NAME, 'total-news-cnt').text))
        except:
            sleep(self.wait_time)
            self.total_cnt = int(re.sub(',','',self.driver.find_element(By.CLASS_NAME, 'total-news-cnt').text))
        self.total_page = math.ceil(self.total_cnt/10)
        print("총 데이터 개수 : ",self.total_cnt)
        print("총 페이지 개수 : " ,self.total_page)

    # def pages(self):
    #     '''
    #         모든 페이지를 크롤링
    #     '''
    #     lastPage = self.driver.find_element(By.CLASS_NAME, 'lastNum').text
    #     # 현재 페이지
    #     isLastPage = False
    #     curPage=1
    #     while(not isLastPage):

    #         ## 페이지 크롤링
    #         self.page_crawling_news()

    #         if curPage == int(lastPage):
    #             isLastPage = True
    #         else:
    #             self.driver.find_element(By.CLASS_NAME, 'page-next.page-link').click()
    #             sleep(self.click_time)
    #             curPage+=1

    def pages(self):
        '''
        모든 페이지를 크롤링
        '''
        lastPage = self.driver.find_element(By.CLASS_NAME, 'lastNum').text
        isLastPage = False
        curPage = 1
        while not isLastPage:
            # 현재 페이지의 뉴스 크롤링
            self.page_crawling_news()

            if curPage >= int(lastPage):
                isLastPage = True
            else:
                try:
                    # 다음 페이지로 넘어가기
                    next_button = self.driver.find_element(By.CLASS_NAME, 'page-next.page-link')
                    self.driver.execute_script("arguments[0].scrollIntoView();", next_button)
                    self.driver.execute_script("arguments[0].click();", next_button)
                    sleep(self.click_time)
                    curPage += 1
                except Exception as e:
                    print("페이지 처리 중 오류 발생:", e)
                    break

    def page_crawling_news(self):
        # 기사 내용 크롤링 로직
        articles = self.driver.find_elements(By.CLASS_NAME, 'news-item')
        for i in range(1, len(articles) + 1):
            # 각 기사에 대한 크롤링 로직
            try:
                self.driver.find_element(By.XPATH, f'//*[@id="news-results"]/div[{i}]/div/div[2]/a/div/strong/span').click();sleep(self.click_time)
            except:
                sleep(self.wait_time)
                self.driver.find_element(By.XPATH, f'//*[@id="news-results"]/div[{i}]/div/div[2]/a/div/strong/span').click();sleep(self.click_time)            

            try:
                date = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul').text;sleep(self.click_time)
            except:
                sleep(self.click_time)
                date = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul').text;sleep(self.click_time)
            
            try:
                title = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/h1').text;sleep(self.click_time)
            except:
                sleep(self.click_time)
                title = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/h1').text;sleep(self.click_time)
                
            try:
                content = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]').text;sleep(self.click_time)
            except:
                sleep(self.click_time)
                content = self.driver.find_element(By.XPATH, f'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]').text;sleep(self.click_time)
            
            self.Crawling_news["Date"].append(date)
            self.Crawling_news["Title"].append(title)
            self.Crawling_news["Content"].append(content)

            try:
                self.driver.find_element(By.XPATH, '//*[@id="news-detail-modal"]/div/div/button').click();sleep(self.click_time)
            except:
                sleep(self.wait_time)
                self.driver.find_element(By.XPATH, '//*[@id="news-detail-modal"]/div/div/button').click();sleep(self.click_time)            
            
    def display(self):
        '''
        기본 정보
        '''
        print("키워드 : ",self.keyword,", 기간 : {} ~ {}".format(self.start_day,self.end_day))
        print("총 페이지 수 : ",self.total_page,"총 데이터 수 : ",self.total_cnt)
    
    def Save(self):
        '''
        
        데이터 저장
        
        '''
        
        # 디렉토리 생성 확인
        if not os.path.exists("./빅카인즈"):
            os.makedirs("./빅카인즈")    
        s_d  = self.start_day[:4]+self.start_day[5:7]+self.start_day[-2:]
        e_d  = self.end_day[:4]+self.end_day[5:7]+self.end_day[-2:]
        pd.DataFrame(self.Crawling_news).to_csv("./빅카인즈/{}_{}_{}.csv".format(self.keyword,s_d,e_d),index=False)



In [3]:
from datetime import datetime
current_date = datetime.now().strftime("%Y-%m-%d")

# 크롤링 조건 설정
keyword = "(무안군 OR 목포시 OR 신안군) AND (교육 OR 인재 OR 학교)"
start_day = "2021-01-01"
end_day = current_date # 현재 날짜
url = "https://www.bigkinds.or.kr/"
# 크롤링 실행
try:
    Search = Search_BigKinds()
    Search.Search_keyword(keyword, start_day, end_day)
    try:
        Search.pages()
    except Exception as e:
        print("페이지 처리 중 오류 발생:", e)
        # 필요한 경우 추가 오류 처리
    Search.Save()
    Search.display()
    Search.driver.close()
except Exception as e:
    print("크롤링 실행 중 오류 발생:", e)
    # 필요한 경우 추가 오류 처리

총 데이터 개수 :  4238
총 페이지 개수 :  424
페이지 처리 중 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="news-results"]/div[9]/div/div[2]/a/div/strong/span"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00DD72A3+45731]
	(No symbol) [0x00D62D51]
	(No symbol) [0x00C5880D]
	(No symbol) [0x00C8B940]
	(No symbol) [0x00C8BE0B]
	(No symbol) [0x00CBD1F2]
	(No symbol) [0x00CA8024]
	(No symbol) [0x00CBB7A2]
	(No symbol) [0x00CA7DD6]
	(No symbol) [0x00C831F6]
	(No symbol) [0x00C8439D]
	GetHandleVerifier [0x010E0716+3229462]
	GetHandleVerifier [0x011284C8+3523784]
	GetHandleVerifier [0x0112214C+3498316]
	GetHandleVerifier [0x00E61680+611968]
	(No symbol) [0x00D6CCCC]
	(No symbol) [0x00D68DF8]
	(No symbol) [0x00D68F1D]
	(No symbol) [0x00D5B2C7]
	BaseThreadInitThunk [0x763C7BA9+25]
	RtlInitial